## 날짜 데이터 생성
+ 작성: 김지훈

- 2012년 1월 1일부터 2050년 12월 31일까지 데이터를 생성합니다.
- 데이터의 표현은 yyyymm (년월) 입니다.
- datetype에 있는 값은 각 월의 마지막 날짜 [ex) 2012년 1월 31일, 2012년 11월 30일 ... ]

In [1]:
import pandas as pd
import numpy as np

In [2]:
start_date='20120101'
end_date='20501231'
df = pd.DataFrame({'Date':pd.date_range(start=start_date, end=end_date, freq='M')}) #월 단위로 데이터 받음
df['Date'] = df['Date'].dt.strftime('%Y%m%d').astype(int)
df

,Date
0,20120131
1,20120229
2,20120331
3,20120430
4,20120531
...,...
463,20500831
464,20500930
465,20501031
466,20501130


## 날짜 데이터 전처리

분기
- 4분기 : 1분기 (1 ~ 3월) / 2분기 (4 ~ 6월) / 3분기 (7 ~ 9월) / 4분기 (10 ~ 12월)  
- 4분기 : 1분기 (1 ~ 2월) / 2분기 (3 ~ 4월) / 3분기 (5 ~ 6월) / 4분기 (7 ~ 8월) / 5분기 (9 ~ 10월) / 6분기 (11 ~ 12월) 

계절 구분
- season_A : 봄(3 ~ 5월) / 여름(6 ~ 8월) / 가을(9 ~ 11월) / 겨울(12 ~ 2월)  
- season_B : 동절기(12월 ~ 3월) / 하절기(6 ~ 9월) / 기타(4 ~ 5월, 10 ~ 11월)  
- season_C : 동절기(10월 ~ 4월) / 하절기(5 ~ 9월)

In [3]:
df_date = pd.DataFrame() #임의의 데이터프레임 생성
df_date['date_id'] = df['Date'].astype(str) #데이터프레임명 변경
df_date['yyyymm']=df_date['date_id'].str[:6]
df_date['date_type'] = pd.to_datetime(df_date['date_id']) #날짜 형식의 데이터 임시 생성
df_date['year'] = df_date['date_type'].dt.year #년
df_date['month'] = df_date['date_type'].dt.month #월
df_date['day'] = df_date['date_type'].dt.day #일
#df_date=df_date.groupby.df_date['']
df_date['quarter_4'] = df_date['date_type'].dt.quarter #분기(4)
df_date['quarter_6'] = np.ceil(df_date['month']/2).astype(int) #분기(6)
df_date['quarter_4'] = df_date['quarter_4'].astype(str)
df_date['quarter_6'] = df_date['quarter_6']

df_date['quarter_4']=df_date['quarter_4'].astype(str)+'/4'
df_date['quarter_6']=df_date['quarter_6'].astype(str)+'/6'

df_date= df_date.groupby(df_date['yyyymm']).max()

df_date['date_id']=df_date.index
#season_A : 봄(3~5월) / 여름(6~8월) / 가을(9~11월) / 겨울(12~2월) 
df_date.loc[(df_date['month'] == 3) | (df_date['month'] == 4)| (df_date['month'] == 5), 'seanson_A'] = '봄'
df_date.loc[(df_date['month'] == 6) | (df_date['month'] == 7)| (df_date['month'] == 8), 'seanson_A'] = '여름'
df_date.loc[(df_date['month'] == 9) | (df_date['month'] == 10)| (df_date['month'] == 11), 'seanson_A'] = '가을'
df_date.loc[(df_date['month'] == 1) | (df_date['month'] == 2)| (df_date['month'] == 12), 'seanson_A'] = '겨울'
#season_B : 동절기(12월~3월) / 하절기(6~9월) / 기타(4~5월, 10~11월)
df_date.loc[(df_date['month'] == 12) | (df_date['month'] == 1)| (df_date['month'] == 2)| (df_date['month'] == 3), 'seanson_B'] = '동절기'
df_date.loc[(df_date['month'] == 6) | (df_date['month'] == 7)| (df_date['month'] == 8)| (df_date['month'] == 9), 'seanson_B'] = '하절기'
df_date.loc[(df_date['month'] == 4) | (df_date['month'] == 5)| (df_date['month'] == 10)| (df_date['month'] == 11), 'seanson_B'] = '기타'
#season_C : 동절기(10월~4월) / 하절기(5~9월)
df_date.loc[(df_date['month'] == 10) | (df_date['month'] == 11)| (df_date['month'] == 12)| (df_date['month'] == 1)
            | (df_date['month'] == 2)| (df_date['month'] == 3)| (df_date['month'] == 4),'seanson_C'] = '동절기'
df_date.loc[(df_date['month'] == 5) | (df_date['month'] == 6)| (df_date['month'] == 7)
            | (df_date['month'] == 8)| (df_date['month'] == 9), 'seanson_C'] = '하절기'
df_date.reset_index(inplace = True,drop=True)

df_date

,date_id,date_type,year,month,day,quarter_4,quarter_6,seanson_A,seanson_B,seanson_C
0,201201,2012-01-31,2012,1,31,1/4,1/6,겨울,동절기,동절기
1,201202,2012-02-29,2012,2,29,1/4,1/6,겨울,동절기,동절기
2,201203,2012-03-31,2012,3,31,1/4,2/6,봄,동절기,동절기
3,201204,2012-04-30,2012,4,30,2/4,2/6,봄,기타,동절기
4,201205,2012-05-31,2012,5,31,2/4,3/6,봄,기타,하절기
...,...,...,...,...,...,...,...,...,...,...
463,205008,2050-08-31,2050,8,31,3/4,4/6,여름,하절기,하절기
464,205009,2050-09-30,2050,9,30,3/4,5/6,가을,하절기,하절기
465,205010,2050-10-31,2050,10,31,4/4,5/6,가을,기타,동절기
466,205011,2050-11-30,2050,11,30,4/4,6/6,가을,기타,동절기


## DB에 데이터 삽입

In [4]:
from sqlalchemy import create_engine
from urllib.parse import quote
import sqlalchemy
import pymysql

In [5]:
user = "kjh" #사용자 이름 (수정)
pwd = "1234" #비밀번호 (수정)
host = "130.1.112.100" #호스트명/IP (수정)
port = 3306 #포트번호 (고정값)
db= "mdw" #사용할 데이터베이스 (필요시 수정)

db_url = f'mysql+pymysql://{user}:{quote(pwd)}@{host}:{port}/{db}' #경로 설정

# MySQL Connector using pymysql
pymysql.install_as_MySQLdb()
engine = create_engine(db_url,encoding='utf-8')
conn = engine.connect()

dtypesql_date = {'date_id_ym':sqlalchemy.types.CHAR(6), 
        'date_type':sqlalchemy.types.DATE, 
        'year':sqlalchemy.types.CHAR(4),
        'month':sqlalchemy.types.VARCHAR(2), 
        'day':sqlalchemy.types.VARCHAR(2),
        'quarter_4':sqlalchemy.types.CHAR(3),
        'quarter_6':sqlalchemy.types.CHAR(3),
        'seanson_A':sqlalchemy.types.VARCHAR(2),
        'seanson_B':sqlalchemy.types.VARCHAR(3),
        'seanson_C':sqlalchemy.types.VARCHAR(3)

            }
df_date.to_sql(name='date_ym', con=engine, if_exists='replace', index=False,dtype=dtypesql_date) #데이터를 넣어줍니다.

468

In [6]:
conn.close()